In [1]:
import anndata
import numpy as np
import sys
import torch
sys.path.append('../../../../')
import velovae as vv

In [2]:
dataset = 'Neuron_scNT'
adata = anndata.read_h5ad(f'data/{dataset}_pp.h5ad')
adata.obs['clusters'] = adata.obs['time'].astype(str)

In [ ]:
genes_retain = ['Jun','Fos','Srf','Maff','Bhlhe40','Fosb','Egr4','Egr1','Egr2','Mef2d',\
                'Cebpb','Rfx3','Hspa5','Pou3f1','Rfxap','Klf9','Cebpg','Klf16']
vv.preprocess(adata, n_gene=2000, gene_retain=genes_retain, min_shared_counts=20)
adata.write_h5ad(f'data/{dataset}_pp.h5ad')

In [3]:
model_path_base = f'checkpoints/{dataset}'
figure_path_base = f'figures/{dataset}'
data_path = f'data/velovae/continuous/{dataset}'
gene_plot = ['Fosb','Rfx3','Cebpg','Homer1']

# Informative Time Prior

In [4]:
adata.obs['tprior'] = adata.obs['time'].to_numpy().astype(float)

# Vanilla VAE

In [5]:
figure_path = f'{figure_path_base}/Vanilla'
model_path = f'{model_path_base}/Vanilla'

torch.manual_seed(2022)
np.random.seed(2022)

vanilla_vae = vv.VanillaVAE(adata, 
                            tmax=20, 
                            device='cuda:0',
                            init_method='tprior',
                            init_key='tprior',
                            tprior='tprior')

vanilla_vae.train(adata, plot=False, gene_plot=gene_plot, figure_path=figure_path)
vanilla_vae.save_model(model_path, 'encoder', 'decoder')
vanilla_vae.save_anndata(adata, 'vanilla', data_path, file_name=f'{dataset}.h5ad')

Initialization using prior time.
Estimating ODE parameters...


  0%|          | 0/2000 [00:00<?, ?it/s]

Estimating the variance...


  0%|          | 0/2000 [00:00<?, ?it/s]

Reinitialize the regular ODE parameters based on estimated global latent time.


  0%|          | 0/2000 [00:00<?, ?it/s]

Gaussian Prior.
Using informative time prior.
------------------------- Train a Vanilla VAE -------------------------
*********        Creating Training/Validation Datasets        *********
*********                      Finished.                      *********
*********                 Creating optimizers                 *********
*********                      Finished.                      *********
*********                    Start training                   *********
Total Number of Iterations Per Epoch: 17, test iteration: 32
********* Early Stop Triggered at epoch 249. *********
*********              Finished. Total Time =   0 h :  0 m : 55 s             *********
Final: Train ELBO = 2342.688,           Test ELBO = 2356.040


# VeloVAE

In [6]:
figure_path = f'{figure_path_base}/VeloVAE'
model_path = f'{model_path_base}/VeloVAE'

torch.manual_seed(2022)
np.random.seed(2022)
vae = vv.VAE(adata, 
             tmax=20, 
             dim_z=5, 
             device='cuda:0',
             init_method='tprior',
             init_key='tprior',
             tprior='tprior',
             init_ton_zero=False)

vae.train(adata, plot=False, gene_plot=gene_plot, figure_path=figure_path)

vae.save_model(model_path, 'encoder', 'decoder')
vae.save_anndata(adata, 'velovae', data_path, file_name=f'{dataset}.h5ad')

Initialization using prior time.
Estimating ODE parameters...


  0%|          | 0/2000 [00:00<?, ?it/s]

Estimating the variance...


  0%|          | 0/2000 [00:00<?, ?it/s]

Reinitialize the regular ODE parameters based on estimated global latent time.


  0%|          | 0/2000 [00:00<?, ?it/s]

Gaussian Prior.
Using informative time prior.
--------------------------- Train a VeloVAE ---------------------------
*********        Creating Training/Validation Datasets        *********
*********                      Finished.                      *********
*********                 Creating optimizers                 *********
*********                      Finished.                      *********
*********                    Start training                   *********
*********                      Stage  1                       *********
Total Number of Iterations Per Epoch: 17, test iteration: 32
*********       Stage 1: Early Stop Triggered at epoch 228.       *********
*********                      Stage  2                       *********
Cell-wise KNN Estimation.


  0%|          | 0/3066 [00:00<?, ?it/s]

Percentage of Invalid Sets: 0.034
Average Set Size: 157
Finished. Actual Time:   0 h :  0 m :  2 s
*********       Stage 2: Early Stop Triggered at epoch 328.       *********
*********              Finished. Total Time =   0 h :  1 m : 21 s             *********
Final: Train ELBO = 2388.209,           Test ELBO = 2315.894


# Full VB

In [7]:
figure_path = f'{figure_path_base}/FullVB'
model_path = f'{model_path_base}/FullVB'

torch.manual_seed(2022)
np.random.seed(2022)
full_vb = vv.VAEFullVB(adata, 
                       tmax=20, 
                       dim_z=5, 
                       device='cuda:0',
                       init_method='tprior',
                       init_key='tprior',
                       tprior='tprior',
                       init_ton_zero=False)

full_vb.train(adata, plot=False, gene_plot=gene_plot, figure_path=figure_path)

full_vb.save_model(model_path, 'encoder', 'decoder')
full_vb.save_anndata(adata, 'fullvb', data_path, file_name=f'{dataset}.h5ad')

Initialization using prior time.
Estimating ODE parameters...


  0%|          | 0/2000 [00:00<?, ?it/s]

Estimating the variance...


  0%|          | 0/2000 [00:00<?, ?it/s]

Reinitialize the regular ODE parameters based on estimated global latent time.


  0%|          | 0/2000 [00:00<?, ?it/s]

Gaussian Prior.
Using informative time prior.
--------------------------- Train a VeloVAE ---------------------------
*********        Creating Training/Validation Datasets        *********
*********                      Finished.                      *********
*********                 Creating optimizers                 *********
*********                      Finished.                      *********
*********                    Start training                   *********
*********                      Stage  1                       *********
Total Number of Iterations Per Epoch: 17, test iteration: 32
*********       Stage 1: Early Stop Triggered at epoch 415.       *********
*********                      Stage  2                       *********
Cell-wise KNN Estimation.


  0%|          | 0/3066 [00:00<?, ?it/s]

Percentage of Invalid Sets: 0.053
Average Set Size: 132
Finished. Actual Time:   0 h :  0 m :  2 s
*********              Finished. Total Time =   0 h :  5 m : 51 s             *********
Final: Train ELBO = 2500.582,           Test ELBO = 2307.322


# Branching ODE

In [8]:
figure_path = f'{figure_path_base}/BrODE'
model_path = f'{model_path_base}/BrODE'

torch.manual_seed(2022)
np.random.seed(2022)
brode = vv.BrODE(adata, 'clusters', 'fullvb_time', 'fullvb_z')

brode.print_weight()

brode.train(adata, 'fullvb_time', 'clusters', plot=False, gene_plot=gene_plot, figure_path=figure_path)

brode.save_model(model_path, 'brode')
brode.save_anndata(adata, 'brode', data_path, file_name=f'{dataset}.h5ad')

vv.plot_transition_graph(adata, save=f'{figure_path_base}/transition.png')

Graph Partition
Number of partitions:  1
Computing type-to-type transition probability
Obtaining the MST in each partition
Initialization using type-specific dynamical model.
Estimating ODE parameters...


  0%|          | 0/2000 [00:00<?, ?it/s]

Estimating the variance...


  0%|          | 0/2000 [00:00<?, ?it/s]

       0  120   15   30   60
0    0.0  0.0  1.0  0.0  0.0
120  0.0  0.0  0.0  0.0  1.0
15   0.0  0.0  1.0  0.0  0.0
30   0.0  0.0  1.0  0.0  0.0
60   0.0  0.0  0.0  1.0  0.0
------------------------ Train a Branching ODE ------------------------
       0  120   15   30   60
0    0.0  0.0  1.0  0.0  0.0
120  0.0  0.0  0.0  0.0  1.0
15   0.0  0.0  1.0  0.0  0.0
30   0.0  0.0  1.0  0.0  0.0
60   0.0  0.0  0.0  1.0  0.0
*********        Creating Training/Validation Datasets        *********
*********                      Finished.                      *********
*********                 Creating optimizers                 *********
*********                      Finished.                      *********
*********                    Start training                   *********
Total Number of Iterations Per Epoch: 17, test iteration: 32
*********           Early Stop Triggered at epoch 390.            *********
*********              Finished. Total Time =   0 h :  7 m : 54 s             *****

# Evaluation

In [ ]:
cluster_edges = [('0' , '15'), 
                 ('15', '30'), 
                 ('30', '60'), 
                 ('60', '120')]
vv.post_analysis(adata,
                 'eval',
                 ['Vanilla VAE','VeloVAE','FullVB','BrODE'],
                 ['vanilla','velovae','fullvb','brode'],
                 compute_metrics=True,
                 genes=gene_plot,
                 grid_size=(1,4),
                 plot_type=['all'],
                 save_path=data_path,
                 cluster_edges=cluster_edges)